# Chunks - Word2Vec (LSTM, CNN)

# Imports & Installs

In [ ]:
# !pip install tf-keras --quiet
# !pip install gensim --quiet
# !pip install tensorflow-text --quiet

!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-text==2.15.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.68 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 30.9 MB/s eta 0:00:00


In [ ]:
#import libraries
import pandas as pd
import os
from google.colab import drive

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras

from keras.layers import Embedding, Input, Dense, Lambda
from keras.models import Model
#import keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import sklearn as sk
from sklearn.model_selection import train_test_split
import os
import nltk
from nltk.data import find
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

# import altair as alt
# from altair_data_server import data_server

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# 4-window plot of loss and accuracy for two models, for comparison

def make_plot(axs,
              model_history1,
              model_history2,
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[metric])
            ax1.plot(model_history.history['val_%s' % metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

# Read and Prepare Data

In [ ]:
# Mount drive and read input files
drive.mount('/content/drive')

final_data_folder = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/'
training_data = pd.read_csv(final_data_folder + 'chunk_training_data.csv')
testing_data = pd.read_csv(final_data_folder + 'chunk_testing_data.csv')

training_data.head()
# chunk_highlights = pd.read_csv('/content/drive/MyDrive/266 Final Project/Processed Data/chunk_highlights.csv')
# chunk_highlights['highlight'] = chunk_highlights['highlight'].fillna('')
# chunk_highlights['label'] = chunk_highlights['highlight'].apply(lambda x: 0 if x =='' else 1)
# chunk_highlights.head()

Mounted at /content/drive


,service_id,service_name,doc_type,doc_text,highlight,paraphrase,label
0,1588,Coinbase,CB Payments & COINBASE UK PRIVACY POLICY,"With law enforcement, officials, or other thir...","With law enforcement, officials, or other thir...",This service reserves the right to disclose yo...,1
1,831,Clockify,Terms of Use,AGREEMENT AMENDMENTS COING may revise and upda...,Any changes shall enter into force upon being ...,"When the service wants to change its terms, us...",1
2,1593,ISODME,Privacy Policy,Tracking technologies and Cookies. We use cook...,"We use cookies, beacons, tags, scripts and oth...","The service may use tracking pixels, web beaco...",1
3,185,Netflix,Terms of Use,The member who created the Netflix account and...,You are responsible for updating and maintaini...,"You must provide your legal name, pseudonyms a...",1
4,836,Scratch,"Privacy Policy (February 14, 2019)",Parents and guardians who register their under...,The results of this research are shared with e...,Your personal data is aggregated into statistics,1


In [ ]:
# class1_df = chunk_highlights[chunk_highlights['label'] == 1]
# class0_df = chunk_highlights[chunk_highlights['label'] == 0].sample(n = len(chunk_highlights[chunk_highlights['label'] == 1]))
# balanced_df = pd.concat([class1_df, class0_df])
# balanced_df['label'].value_counts()

In [ ]:
# X = balanced_df['doc_text'].values
# y = balanced_df['label'].values
# # y = balanced_df['highlight'].values

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)
# print(X_train.shape)
# print(X_test.shape)

# LSTM Experiment

In [ ]:
# Tokenize train and test data
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(training_data['doc_text'].values)
test_tokens = tokenizer.tokenize(testing_data['doc_text'].values)

# Take a look at a training example
train_tokens[0]

<tf.Tensor: shape=(153,), dtype=string, numpy=
array([b'With', b'law', b'enforcement,', b'officials,', b'or', b'other',
       b'third', b'parties', b'when', b'we', b'are', b'compelled', b'to',
       b'do', b'so', b'by', b'a', b'subpoena,', b'court', b'order,',
       b'or', b'similar', b'legal', b'procedure,', b'or', b'when', b'we',
       b'believe', b'in', b'good', b'faith', b'that', b'the',
       b'disclosure', b'of', b'personal', b'information', b'is',
       b'necessary', b'to', b'prevent', b'physical', b'harm', b'or',
       b'financial', b'loss,', b'to', b'report', b'suspected', b'illegal',
       b'activity,', b'or', b'to', b'investigate', b'violations', b'of',
       b'our', b'User', b'Agreement', b'or', b'any', b'other',
       b'applicable', b'policies.', b'If', b'you', b'establish', b'a',
       b'CB', b'Account', b'indirectly', b'on', b'a', b'third', b'party',
       b'website', b'or', b'via', b'a', b'third', b'party',
       b'application,', b'any', b'information', b't

In [ ]:
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

wvmodel = KeyedVectors.load_word2vec_format(datapath(word2vec_sample), binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [ ]:
EMBEDDING_DIM = 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(wvmodel) + 1, EMBEDDING_DIM))
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(wvmodel.index_to_key):
    embedding_vector = wvmodel[word]

    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
MAX_SEQUENCE_LENGTH = 256

def docs_to_vocab_ids(tokenized_texts_list, MAX_SEQUENCE_LENGTH):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    text_labels = []
    valid_example_list = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])

        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:MAX_SEQUENCE_LENGTH]
        n_padding = (MAX_SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with unknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding
        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)

        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)
    return np.array(texts_vocab_ids)

In [ ]:
train_input = docs_to_vocab_ids(train_tokens, MAX_SEQUENCE_LENGTH)
test_input = docs_to_vocab_ids(test_tokens, MAX_SEQUENCE_LENGTH)

train_labels = np.array(training_data['label'].values)
test_labels = np.array(testing_data['label'].values)

Examples processed:  0
Examples processed:  5000
Total examples:  9567
Examples processed:  0
Examples processed:  5000
Total examples:  5409


In [ ]:
def build_lstm_model(lstm_units, MAX_SEQUENCE_LENGTH, embeddings_trainable=False):
  lstm_input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
  lstm_embeddings = tf.keras.layers.Embedding(embedding_matrix.shape[0],
                                                    embedding_matrix.shape[1],
                                                    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                                    input_length=MAX_SEQUENCE_LENGTH,
                                                    trainable=embeddings_trainable)(lstm_input_layer)

  lstm_layer = tf.keras.layers.LSTM(units=lstm_units, return_sequences=True, name='lstm_layer')(lstm_embeddings)
  # lstm_layer = tf.keras.layers.Dense(10, activation='relu', name='hidden_layer')(lstm_layer)
  lstm_layer = tf.keras.layers.Dropout(0.2)(lstm_layer)

  lstm_layer = tf.keras.layers.LSTM(units=lstm_units, return_sequences=False, name='lstm_layer')(lstm_embeddings)
  lstm_layer = tf.keras.layers.Dropout(0.2)(lstm_layer)

  lstm_prediction = tf.keras.layers.Dense(1, activation='sigmoid', name='lstm_output_layer')(lstm_layer)

  lstm_model = tf.keras.Model(inputs=lstm_input_layer, outputs=lstm_prediction)
  lstm_model.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.Adam(learning_rate=0.0001,
                                                    beta_1=0.9,
                                                    beta_2=0.999,
                                                    epsilon=1e-07,
                                                    amsgrad=False),
                    metrics=['accuracy'])

  return lstm_model

In [ ]:
tf.keras.backend.clear_session()

lstm_model = build_lstm_model(lstm_units=16, MAX_SEQUENCE_LENGTH=256)
lstm_history =lstm_model.fit(train_input,
                             train_labels,
                             validation_data=(test_input, test_labels),
                             batch_size=16,
                             epochs=3
                            )

Epoch 1/3
598/598 [==============================] - 55s 89ms/step - loss: 0.6931 - accuracy: 0.4954 - val_loss: 0.6932 - val_accuracy: 0.2262
Epoch 2/3
598/598 [==============================] - 47s 78ms/step - loss: 0.6927 - accuracy: 0.4943 - val_loss: 0.6926 - val_accuracy: 0.2307
Epoch 3/3
598/598 [==============================] - 46s 77ms/step - loss: 0.6924 - accuracy: 0.5043 - val_loss: 0.6980 - val_accuracy: 0.2272


LSTM unit size refers to the dimensionality of the output space for each time step. Each time step will output an n-dimensional vector. This vector represents the learned features from the input sequence up to that point.

This hyperparameter controls the "capacity" of the LSTM cell to learn and store information over time. A higher number of units increase the capacity to capture complex patterns, but uses more memory and can lead to overfitting. A lower number of units reduces model complexity, which lowers runtime and overfitting, but may miss patterns in the data.

# CNN Experiment

In [ ]:
def build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate, embeddings_trainable=False):
    cnn_input_layer = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    cnn_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=embeddings_trainable)
    cnn_embeddings = cnn_embedding_layer(cnn_input_layer)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(cnn_embeddings)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    last_hidden_output = keras.layers.Dropout(rate=dropout_rate)(conv_output)

    for dense_layer_dim in dense_layer_dims:
        last_hidden_output = keras.layers.Dense(dense_layer_dim, activation='relu')(last_hidden_output)

    cnn_prediction = keras.layers.Dense(1, activation='sigmoid')(last_hidden_output)

    cnn_model = keras.Model(inputs=cnn_input_layer, outputs=cnn_prediction)
    cnn_model.compile(optimizer='adam',
                      loss='binary_crossentropy',  # From information theory notebooks.
                      metrics=['accuracy'])
    return cnn_model

In [ ]:
# Now specify model hyperparameters and call the function to create a CNN model

num_filters = [100, 100, 50, 25]
kernel_sizes = [3, 4, 5, 10]
dense_layer_dims = [100, 50]
dropout_rate = 0.5

cnn_model = build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate)

In [ ]:
cnn_history = cnn_model.fit(train_input, train_labels,
                            validation_data=(test_input, test_labels),
                            batch_size=32,
                            epochs=5
                            )

Epoch 1/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5290 - loss: 0.6895 - val_accuracy: 0.6405 - val_loss: 0.6575
Epoch 2/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6270 - loss: 0.6439 - val_accuracy: 0.6363 - val_loss: 0.6331
Epoch 3/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7008 - loss: 0.5749 - val_accuracy: 0.7059 - val_loss: 0.5674
Epoch 4/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7658 - loss: 0.4946 - val_accuracy: 0.7380 - val_loss: 0.5277
Epoch 5/5
297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8195 - loss: 0.4057 - val_accuracy: 0.7270 - val_loss: 0.5789
